In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qdov7k3o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qdov7k3o
  Resolved https://github.com/huggingface/transformers to commit ff8b88a948fc2f6aba421ca64ad165291928dcee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.56.0.dev0-py3-none-any.whl size=12585757 sha256=708f6e2a67b5bd20577dd629cf2e4c1b0119fbee47892698f52b3f87ed6c975d
  Stored in directory: /tmp/pip-ephem-wheel-cache-g0e_uz4o/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import os
from tqdm.notebook import tqdm

In [ ]:
# login HF


In [ ]:
src = str(Path.cwd().resolve().parents[0]/ 'src')
sys.path.append(src)
from config.paths import DATA, EXPERIMENTS, METADATA

In [ ]:
# src = str(Path.cwd().resolve().parents[0]/ 'src')
# sys.path.append(src)
# from config.paths import DATA, EXPERIMENTS, METADATA

In [5]:
DATA = '/content/drive/MyDrive/dataHN/data'
EXPERIMENTS = '/content/drive/MyDrive/dataHN/experiments'

In [8]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ImageFeatureDataset(Dataset):
    def __init__(self, img_dir, processor, grayscale=False):
        self.img_dir = img_dir
        self.image_paths = [
            os.path.join(img_dir, f)
            for f in os.listdir(img_dir)
            if f.lower().endswith(('.jpg', '.jpeg'))
        ]
        self.processor = processor
        self.grayscale = grayscale

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)

        if self.grayscale:
            image = image.convert('L').convert('RGB')
        else:
            image = image.convert('RGB')

        inputs = self.processor(images=image, return_tensors="pt")
        return inputs['pixel_values'].squeeze(0), img_path


In [ ]:

model_name = "facebook/dinov3-vitb16-pretrain-lvd1689m"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, device_map="auto")
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

DINOv3ViTModel(
  (embeddings): DINOv3ViTEmbeddings(
    (patch_embeddings): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (rope_embeddings): DINOv3ViTRopePositionEmbedding()
  (layer): ModuleList(
    (0-11): 12 x DINOv3ViTLayer(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attention): DINOv3ViTAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (o_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (layer_scale1): DINOv3ViTLayerScale()
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): DINOv3ViTMLP(
        (up_proj): Linear(in_features=768, out_features=3072, bias=True)
        (down_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act_fn): GELUActivation()
 

# Embeddings couleur (images entières)

In [ ]:
obj_dir_img = os.path.join(DATA, 'processed', 'obj_imgs_cleaned')
obj_output = os.path.join(DATA, 'embeddings', 'dino3_obj_color')


os.makedirs(obj_output, exist_ok=True)
os.makedirs(doc_output, exist_ok=True)


In [ ]:
#Corpus objets
dataset_color = ImageFeatureDataset(img_dir=obj_dir_img, processor=processor)

dataloader = DataLoader(dataset_color, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

print(f"Dataset : {len(dataset_color)} images.")

with torch.inference_mode():
    for pixel_values, image_paths in tqdm(dataloader, desc="Processing Images"):
        pixel_values = pixel_values.to(model.device)

        outputs = model(pixel_values)
        pooled_output = outputs.pooler_output

        for i, output in enumerate(pooled_output):
            original_file_name = os.path.basename(image_paths[i])
            file_name, _ = os.path.splitext(original_file_name)
            output_path = os.path.join(obj_output, file_name + ".pt")
            torch.save(output.cpu(), output_path)

Dataset : 2127 images.


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Processing Images:   0%|          | 0/133 [00:00<?, ?it/s]

In [ ]:
doc_dir_img = os.path.join(DATA, 'cleaned', 'doc_imgs_cleaned')
doc_output = os.path.join(DATA, 'embeddings', 'dino3_doc_color')


os.makedirs(obj_output, exist_ok=True)
os.makedirs(doc_output, exist_ok=True)


In [ ]:
#Corpus docs
dataset_color = ImageFeatureDataset(img_dir=doc_dir_img, processor=processor)

dataloader = DataLoader(dataset_color, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

print(f"Dataset : {len(dataset_color)} images.")

with torch.inference_mode():
    for pixel_values, image_paths in tqdm(dataloader, desc="Processing Images"):
        pixel_values = pixel_values.to(model.device)

        outputs = model(pixel_values)
        pooled_output = outputs.pooler_output

        for i, output in enumerate(pooled_output):
            original_file_name = os.path.basename(image_paths[i])
            file_name, _ = os.path.splitext(original_file_name)
            output_path = os.path.join(doc_output, file_name + ".pt")
            torch.save(output.cpu(), output_path)

Dataset : 4596 images.


Processing Images:   0%|          | 0/288 [00:00<?, ?it/s]

# Embeddings grayscale (sur crops)

## YOLO crops documents

In [ ]:
yolo_doc_dir_img = os.path.join(DATA, 'processed', 'yolo_doc_crops')

yolo_doc_output = os.path.join(DATA, 'embeddings', 'dino3_doc_crops_gray')

os.makedirs(yolo_doc_output, exist_ok=True)

In [14]:
print(len(os.listdir(yolo_doc_dir_img)))

7705


In [18]:
#Corpus docs
dataset_gray = ImageFeatureDataset(img_dir=yolo_doc_dir_img, processor=processor, grayscale=True)

dataloader = DataLoader(dataset_gray, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

print(f"Dataset : {len(dataset_gray)} images.")

Dataset : 7705 images.


In [21]:
with torch.inference_mode():
    for pixel_values, image_paths in tqdm(dataloader, desc="Processing Images"):
        pixel_values = pixel_values.to(model.device)

        outputs = model(pixel_values)
        pooled_output = outputs.pooler_output

        for i, output in enumerate(pooled_output):
            original_file_name = os.path.basename(image_paths[i])
            file_name, _ = os.path.splitext(original_file_name)
            output_path = os.path.join(yolo_doc_output, file_name + ".pt")
            torch.save(output.cpu(), output_path)

Processing Images:   0%|          | 0/482 [00:00<?, ?it/s]

## Crops objets

In [ ]:
obj_dir_img = os.path.join(DATA, 'processed', 'g-dino_obj_crops')

obj_output = os.path.join(DATA, 'embeddings', 'dino3_obj_crops_gray')

os.makedirs(yolo_doc_output, exist_ok=True)

In [24]:
#Corpus objets
dataset_gray = ImageFeatureDataset(img_dir=obj_dir_img, processor=processor, grayscale = True)

dataloader = DataLoader(dataset_gray, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

print(f"Dataset : {len(dataset_gray)} images.")

Dataset : 2127 images.


In [25]:
with torch.inference_mode():
    for pixel_values, image_paths in tqdm(dataloader, desc="Processing Images"):
        pixel_values = pixel_values.to(model.device)

        # Get the model's output
        outputs = model(pixel_values)
        pooled_output = outputs.pooler_output

        # Save each individual feature vector in the batch
        for i, output in enumerate(pooled_output):
            original_file_name = os.path.basename(image_paths[i])
            file_name, _ = os.path.splitext(original_file_name)
            output_path = os.path.join(obj_output, file_name + ".pt")
            torch.save(output.cpu(), output_path)

Processing Images:   0%|          | 0/133 [00:00<?, ?it/s]